# Sudoku miniproject

Below you will find the most elegant solution of sudoku solver coded by Peter Norvig. It's a good notation and modified implementation of four techniques that are enough to solve tough sudoku.

You can find the original post here: http://norvig.com/sudoku.html

The best website about sudoku: http://www.sudokudragon.com/sudokutheory.htm

### Sudoku Notation and Preliminary Notions

First we have to agree on some notation. A Sudoku puzzle is a grid of 81 squares; the majority of enthusiasts label the columns 1-9, the rows A-I, and call a collection of nine squares (column, row, or box) a unit and the squares that share a unit the peers. A puzzle leaves some squares blank and fills others with digits, and the whole idea is:
A puzzle is solved if the squares in each unit are filled with a permutation of the digits 1 to 9.
That is, no digit can appear twice in a unit, and every digit must appear once. This implies that each square must have a different value from any of its peers. Here are the names of the squares, a typical puzzle, and the solution to the puzzle:

```
 A1 A2 A3| A4 A5 A6| A7 A8 A9    4 . . |. . . |8 . 5     4 1 7 |3 6 9 |8 2 5 
 B1 B2 B3| B4 B5 B6| B7 B8 B9    . 3 . |. . . |. . .     6 3 2 |1 5 8 |9 4 7
 C1 C2 C3| C4 C5 C6| C7 C8 C9    . . . |7 . . |. . .     9 5 8 |7 2 4 |3 1 6 
---------+---------+---------    ------+------+------    ------+------+------
 D1 D2 D3| D4 D5 D6| D7 D8 D9    . 2 . |. . . |. 6 .     8 2 5 |4 3 7 |1 6 9 
 E1 E2 E3| E4 E5 E6| E7 E8 E9    . . . |. 8 . |4 . .     7 9 1 |5 8 6 |4 3 2 
 F1 F2 F3| F4 F5 F6| F7 F8 F9    . . . |. 1 . |. . .     3 4 6 |9 1 2 |7 5 8 
---------+---------+---------    ------+------+------    ------+------+------
 G1 G2 G3| G4 G5 G6| G7 G8 G9    . . . |6 . 3 |. 7 .     2 8 9 |6 4 3 |5 7 1 
 H1 H2 H3| H4 H5 H6| H7 H8 H9    5 . . |2 . . |. . .     5 7 3 |2 9 1 |6 8 4 
 I1 I2 I3| I4 I5 I6| I7 I8 I9    1 . 4 |. . . |. . .     1 6 4 |8 7 5 |2 9 3 
```

Every square has exactly 3 units and 20 peers. For example, here are the units and peers for the square C2:

```
    A2   |         |                    |         |            A1 A2 A3|         |         
    B2   |         |                    |         |            B1 B2 B3|         |         
    C2   |         |            C1 C2 C3| C4 C5 C6| C7 C8 C9   C1 C2 C3|         |         
---------+---------+---------  ---------+---------+---------  ---------+---------+---------
    D2   |         |                    |         |                    |         |         
    E2   |         |                    |         |                    |         |         
    F2   |         |                    |         |                    |         |         
---------+---------+---------  ---------+---------+---------  ---------+---------+---------
    G2   |         |                    |         |                    |         |         
    H2   |         |                    |         |                    |         |         
    I2   |         |                    |         |                    |         |    
```

In [1]:
#notation
import time
import random
import shutil
import sys

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]

digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
squares  = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
units = dict((s, [u for u in unitlist if s in u]) 
             for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in squares)

### Sudoku grid
Now that we have squares, units, and peers, the next step is to define the Sudoku playing grid. Actually we need two representations: First, a textual format used to specify the initial state of a puzzle; we will reserve the name grid for this. Second, an internal representation of any state of a puzzle, partially solved or complete; this we will call a values collection because it will give all the remaining possible values for each square. For the textual format (grid) we'll allow a string of characters with 1-9 indicating a digit, and a 0 or period specifying an empty square. All other characters are ignored (including spaces, newlines, dashes, and bars). So each of the following three grid strings represent the same puzzle:



In [2]:
# parser
def grid_values(grid):
    "Convert grid into a dict of {square: char} with '0' or '.' for empties."
    chars = [c for c in grid if c in digits or c in '0.']
    assert len(chars) == 81
    return dict(zip(squares, chars))

def display(values):
    "Display these values as a 2-D grid."
    width = 1+max(len(values[s]) for s in squares)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '') for c in cols))
        if r in 'CF': print(line)
    print()

### Parser
Now for values. One might think that a 9 x 9 array would be the obvious data structure. But squares have names like 'A1', not (0,0). Therefore, values will be a dict with squares as keys. The value of each key will be the possible digits for that square: a single digit if it was given as part of the puzzle definition or if we have figured out what it must be, and a collection of several digits if we are still uncertain. This collection of digits could be represented by a Python set or list, but I chose instead to use a string of digits (we'll see why later). So a grid where A1 is 7 and C7 is empty would be represented as {'A1': '7', 'C7': '123456789', ...}.

In [3]:
def parse_grid(values):
    possible_grid = dict((square, digits) for square in values)
    for square in values:
        if values[square] in digits:
            possible_grid[square] = values[square]

    return possible_grid

# Constraint Propagation - refactored code
Those with experience solving Sudoku puzzles know that there are two important strategies that we can use to make progress towards filling in all the squares:

(1) If a square has only one possible value, then eliminate that value from the square's peers - eliminate(values).

(2) If a unit has only one possible place for a value, then put the value there - only_choice(values)

As an example of strategy (1) if we assign 7 to A1, yielding {'A1': '7', 'A2':'123456789', ...}, we see that A1 has only one value, and thus the 7 can be removed from its peer A2 (and all other peers), giving us {'A1': '7', 'A2': '12345689', ...}. As an example of strategy (2), if it turns out that none of A3 through A9 has a 3 as a possible value, then the 3 must belong in A2, and we can update to {'A1': '7', 'A2':'3', ...}. These updates to A2 may in turn cause further updates to its peers, and the peers of those peers, and so on. This process is called constraint propagation.

In [4]:
def eliminate(values):
    """
    Iterate through all squares and every time 
       if there is a square with one value, 
       then eliminate this value from the peers

    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    for square in values:
        if len(values[square]) == 1:
            for peer in peers[square]:
                values[peer] = values[peer].replace(values[square],'')
    return values


def only_choice(values):
    """
    Iterate through all squares and every time
        if there is a square with a value that only fits in one square, 
        assign the value to this square

    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    for square in values:
        for digit in values[square]:
            if len(values[square]) > 1:
                for unit in units[square]:
                    possibilities = ''
                    for unit_square in unit:
                        if unit_square != square:
                            possibilities += values[unit_square]
                    if digit not in possibilities:
                        values[square] = digit
                        break
                   
    return values


def reduce_puzzle(values):
    """
    Reduce sudoku using eliminate() and only_choice()
    
    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    old_values = dict.fromkeys(values.keys())
    while old_values != values:
        old_values = dict(values)
        
        eliminate(values)
        
        only_choice(values)
            
    return values

# Sub-group exclusion
http://www.sudokudragon.com/sudokustrategy.htm#XL2100

In [5]:
def check_possibilities(values, unit):
    """
    Check all the possibilities of squares inside the unit.
    
    input: A sudoku in dictionary form, a unit to be checked.
    output: The resulting possibilities string.
    """
    possibilities = ''
    for unit_square in unit:
        if len(values[unit_square]) > 1:
            possibilities += values[unit_square]
            
    return possibilities


def double_digits(possibilities):
    """
    Look for digits which appear only two times.
    
    input: Possibilites from squares of the unit in string.
    output: Double digits string
    """
    twin_digits = ''
    for digit in digits:
        if possibilities.count(digit) == 2:
            twin_digits += digit
            
    return twin_digits


def subgroup_exclusion(values):
    """
    Eliminate values using the sub-group exclusion strategy
    
    input: A sudoku in dictionary form.
    output: The resulting sudoku in dictionary form.
    """
    for square in values:
        #Consider only unsolved squares 
        if len(values[square]) > 1:
            #Iterate through units of the square
            for unit in units[square]:

                possibilities = check_possibilities(values,unit)

                twin_digits = double_digits(possibilities)

                for digit in twin_digits:
                    if digit in values[square]:
                        #twin_unit -> another unit with the considered square
                        for twin_unit in units[square]:
                            if twin_unit != unit:
                                #twin_square -> another square in both the twin_unit and the unit with the digit
                                for twin_square in twin_unit:
                                    if twin_square != square and twin_square in unit and digit in values[twin_square]:
                                        #exclude_square -> square to eliminate the digit from
                                        for exclude_square in twin_unit:
                                            if exclude_square != square and exclude_square != twin_square:
                                                values[exclude_square] = values[exclude_square].replace(digit,'')
 
    return values

# Hidden twins
http://www.sudokudragon.com/guidehiddentwins.htm

In [6]:
def hidden_twins(values):
    """
    Eliminate values using the sub-group hidden twins strategy
    
    input: A sudoku in dictionary form.
    output: The resulting sudoku in dictionary form.
    """
    for square in values:
        #Consider only unsolved squares 
        if len(values[square]) > 1:
            #Iterate through units of the square
            for unit in units[square]:

                possibilities = check_possibilities(values,unit)

                twin_digits = double_digits(possibilities)
                        
                #Look for hidden twins
                #Iterate through two different twin_digits
                for digit1 in twin_digits:
                    for digit2 in twin_digits:
                        if digit1 != digit2 and digit1 in values[square] and digit2 in values[square]:
                            #twin_square -> another square with hidden twins
                            for twin_square in unit:
                                if twin_square != square and digit1 in values[twin_square] and digit2 in values[twin_square]:
                                    values[square] = digit1 + digit2
                                    values[twin_square] = digit1 + digit2

    return values

# Solve sudoku

In [7]:
def solve(values):
    """
    Solve a sudoku using constraint propagation, sub-group exclusion and hidden twins techniques
    
    input: A sudoku in dictionary form.
    output: The resulting sudoku in dictionary form.
    """
    old_values = dict.fromkeys(values.keys())
    #Stop a loop when there is not reduction of possibilities
    while old_values != values:
        old_values = dict(values)
        
        reduce_puzzle(values)
        
        subgroup_exclusion(values)
        
        hidden_twins(values)
            
    return values

# Search
http://norvig.com/sudoku.html

In [8]:
def search(values):
    # Make a copy of sudoku
    values_copy = dict(values)
    
    # Check if sudoku is solved
    if all(len(values_copy[s]) == 1 for s in values_copy):
        return values_copy
    
    # Search for the square with the lowest number of digits
    min_square = 'A1'
    for square in values_copy:
        if len(values_copy[min_square]) < 2:
            min_square = square
        if len(values_copy[square]) > 1 and len(values_copy[min_square]) > len(values_copy[square]):
            min_square = square

    for digit in values_copy[min_square]:
        values_copy[min_square] = digit
        solve(values_copy)
        return search(values_copy)

# Generate hard sudoku

In [9]:
def generate():
    grid = '.................................................................................'
    sudoku_values = parse_grid(grid_values(grid))
    sudoku_values_copy = dict(sudoku_values)
    
    keys = list(sudoku_values.keys())
    random.shuffle(keys)
    
    no_digits = 0
    digits_occured = ''
    finished = False
    while not finished:
        sudoku_values = parse_grid(grid_values(grid))
        sudoku_values_copy = dict(sudoku_values)
        
        for key in keys:
            restart = True
            while restart:
                digit = str(random.randrange(1,10))
                if digit in sudoku_values_copy[key]:
                    sudoku_values[key] = digit
                    sudoku_values_copy[key] = digit
                    restart = False
                    no_digits += 1

                    if digit not in digits_occured:
                        digits_occured += digit
            solve(sudoku_values_copy)

            if no_digits > 16 and len(digits_occured) > 7:
                finished = True
                break
                
    generated_grid = ''
    for s,i in zip(sudoku_values,range(len(grid))):
        if len(sudoku_values[s]) == 1:
            generated_grid += sudoku_values[s]
        else:
            generated_grid += '.'    
            
    return generated_grid           

# Generate 200 sudoku, choose 10 hardest

In [10]:
# # Generate
# N = 200
# sudoku_time_file = open('sudoku_time.txt', 'w')
# for i in range(N):
#     file = open('./sudoku/sudoku' + str(i) + '.txt', 'w')
#     sudoku_grid = generate()
#     file.write(sudoku_grid)
#     file.close()
    
#     sudoku = grid_values(sudoku_grid)
#     sudoku_values = parse_grid(sudoku)

#     start = time.time()
#     solve(sudoku_values)
#     try:
#         sudoku_solved = search(sudoku_values)
#     except RecursionError:
#         sudoku_time_file.write('{:3f}\n'.format(0))
#         continue
    
#     sudoku_time_file.write('{:3f}\n'.format(time.time() - start))
    
# sudoku_time_file.close()

# # Choose hardest
# K = 10
# tab_max = []
# max_time = 0
# for i in range(K):
#     file = open('sudoku_time.txt', 'r')
#     j = 0
#     j_max = 0
#     max_time = 0
#     for line in file:
#         if float(line) > max_time and j not in tab_max:
#             max_time = float(line)
#             j_max = j
#         j += 1
#     tab_max.append(j_max)
#     file.close()

# for i in tab_max:
#     shutil.copyfile('./sudoku/sudoku' + str(i) + '.txt', './hard_sudoku/sudoku' + str(i) + '.txt')

# Read sudoku from file

In [11]:
file = open('./hard_sudoku/sudoku116.txt', 'r')
sudoku_grid = file.read()
file.close()

sudoku = grid_values(sudoku_grid)
print('Sudoku puzzle')
display(sudoku)
sudoku_values = parse_grid(sudoku)

start = time.time()
solve(sudoku_values)
try:
    sudoku_solved = search(sudoku_values)
except RecursionError:
    sys.exit("Couldn't solve sudoku :(")
    
print('Sudoku solved in {:3f}'.format(time.time() - start))
display(sudoku_solved)

Sudoku puzzle
. . . |. . . |. 5 . 
. . . |. . . |. . . 
. . . |. . 6 |. . . 
------+------+------
. . . |. 9 8 |4 2 . 
1 . . |3 . . |. . . 
. . 4 |. . 7 |. . . 
------+------+------
. . . |. . . |. . . 
. . . |7 . . |. . . 
2 1 6 |. . 9 |. 8 7 

Sudoku solved in 0.116335
7 8 9 |1 3 2 |6 5 4 
3 6 2 |9 4 5 |1 7 8 
4 5 1 |8 7 6 |9 3 2 
------+------+------
5 7 3 |6 9 8 |4 2 1 
1 9 8 |3 2 4 |7 6 5 
6 2 4 |5 1 7 |8 9 3 
------+------+------
8 3 7 |2 6 1 |5 4 9 
9 4 5 |7 8 3 |2 1 6 
2 1 6 |4 5 9 |3 8 7 

